In [41]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

def scrape_vnexpress_news():
    url = 'https://vnexpress.net/tin-tuc-24h'
    response = requests.get(url)
    
    if response.status_code != 200:
        print("Failed to retrieve the webpage")
        return
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    articles = []
    
    for item in soup.find_all('article', class_='item-news'):
        title = item.find('h3', class_='title-news')
        link = item.find('a', href=True)
        summary = item.find('p', class_='description')
        
        if title and link and summary:
            full_link = link['href']
            if not full_link.startswith('http'):
                full_link = 'https://vnexpress.net' + full_link
            articles.append({
                'title': title.get_text(strip=True),
                'link': full_link,
                'summary': summary.get_text(strip=True),
                'label': 0
            })
    
    return articles

In [42]:
# Lấy dữ liệu từ trang VNExpress
articles = scrape_vnexpress_news()
for article in articles:
    print(article)

{'title': 'Mỗi tín chỉ carbon lúa của Việt Nam được định giá 20 USD', 'link': 'https://vnexpress.net/moi-tin-chi-carbon-lua-cua-viet-nam-duoc-dinh-gia-20-usd-4782257.html', 'summary': 'Bộ Nông nghiệp và Phát triển Nông thôn đang định giá mỗi tín chỉ carbon lúa ở mức 20 USD, cao hơn nhiều so với mức giá dự tính của Ngân hàng Thế giới.', 'label': 0}
{'title': 'Trung tá công an hy sinh khi khống chế người cầm dao được truy tặng bằng Tổ quốc ghi công', 'link': 'https://vnexpress.net/trung-ta-cong-an-hy-sinh-khi-khong-che-nguoi-cam-dao-duoc-truy-tang-bang-to-quoc-ghi-cong-4782210.html', 'summary': 'Thừa Thiên - HuếTrung tá Trần Duy Hùng, người hy sinh khi làm nhiệm vụ, được Chính phủ truy tặng bằng Tổ quốc ghi công, ngày 16/8.', 'label': 0}
{'title': 'Đàn ông sẽ trở nên yếu đuối nếu thiếu chút gia trưởng và độc đoán', 'link': 'https://vnexpress.net/dan-ong-se-tro-nen-yeu-duoi-neu-thieu-chut-gia-truong-va-doc-doan-4782241.html', 'summary': 'Nếu tiếp tục sợ vợ, cả đời này bạn không bao giờ đư

In [44]:
def store_articles_to_mongodb(articles):
    try:
        # Kết nối đến MongoDB (Localhost hoặc sử dụng URI cho MongoDB trên cloud)
        client = MongoClient('mongodb://localhost:27017')
        
        # Tạo một cơ sở dữ liệu mới hoặc kết nối đến một cơ sở dữ liệu đã có
        db = client['news_database']
        
        # Tạo một collection mới hoặc kết nối đến một collection đã có
        collection = db['VNExpress_news']
        
        # Chèn các bài báo vào collection
        if articles:
            collection.insert_many(articles)
            print(f"Lưu {len(articles)} tin tức vào MongoDB.")
        else:
            print("Không có tin tức nào được lưu")
        
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        # Đóng kết nối
        client.close()

In [45]:
articles = scrape_vnexpress_news()
if articles:
    store_articles_to_mongodb(articles)

Lưu 30 tin tức vào MongoDB.
